---
title: "2. Generate credit card transactions data and send to kafka topic"
date: 2021-04-25
type: technical_note
draft: false
---

# Generate credit card transactions data and send to kafka topic.

### Inspiration of this example was taken from [here](https://github.com/aws-samples/amazon-sagemaker-feature-store-streaming-aggregation).

#### Prerequisites 

In [1]:
#!pip install Faker

#### Imports 

In [2]:
import os
import bisect
import datetime
import hashlib
import math
import random

from collections import defaultdict
from typing import Optional, Union, Any, Dict, List, TypeVar, Tuple

import pandas as pd
import numpy as np

from faker import Faker

In [3]:
# Seed for Reproducibility
faker = Faker()
faker.seed_locale('en_US', 0)

In [4]:
SEED = 12345
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

#### Constants 

In [5]:
TOTAL_UNIQUE_USERS = 2000
TOTAL_UNIQUE_TRANSACTIONS = 54000
CASH_WITHRAWAL_CARDS_TOTAL = 1000 
TOTAL_UNIQUE_CASH_WITHDRAWALS = 12000 
ATM_WITHRAWAL_SEQ_LENGTH = [3, 4, 5, 6, 7, 8, 9, 10]
NORMAL_ATM_RADIUS = 0.01
START_DATE = '2022-01-01 00:00:00'
END_DATE = '2022-03-01 00:00:00'
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'

AMOUNT_DISTRIBUTION_PERCENTAGES = {
                                   0.05: (0.01, 1.01), 
                                   0.075: (1, 11.01),
                                   0.525: (10, 100.01),
                                   0.25: (100, 1000.01),
                                   0.099: (1000, 10000.01),
                                   0.001: (10000, 30000.01)
                                  }

CATEGORY_PERC_PRICE = {
                       "Grocery":              (0.5, 0.01, 100), 
                       "Restaurant/Cafeteria": (0.2, 1, 100),
                       "Health/Beauty":        (0.1, 10, 500.01),
                       "Domestic Transport":   (0.1, 10, 100.01),
                       "Clothing":             (0.05, 10, 2000.01),
                       "Electronics":          (0.02, 100, 10000.01),
                       "Sports/Outdoors":      (0.015, 10, 100.01),
                       "Holliday/Travel":      (0.014, 10, 100.01),              
                       "Jewelery":             (0.001, 10, 100.01)
                       }

FRAUD_RATIO = 0.0025 # percentage of transactions that are fraudulent
NUMBER_OF_FRAUDULENT_TRANSACTIONS = int(FRAUD_RATIO * TOTAL_UNIQUE_TRANSACTIONS)
ATTACK_CHAIN_LENGTHS = [3, 4, 5, 6, 7, 8, 9, 10]

### Generate Transactions

#### Generate Unique Credit Card Numbers 
<p> Credit card numbers are uniquely assigned to users. Since, there are 10K users, we would want to generate 10K unique card numbers.</p>

In [6]:
def generate_unique_credit_card_numbers(n: int) -> list:
    cc_ids = set()
    for _ in range(n):
        cc_id = faker.credit_card_number(card_type='visa')
        cc_ids.add(cc_id)
    return list(cc_ids) 

In [7]:
credit_card_numbers = generate_unique_credit_card_numbers(TOTAL_UNIQUE_USERS)

In [8]:
assert len(credit_card_numbers) == TOTAL_UNIQUE_USERS 
assert len(credit_card_numbers[0]) == 16 # validate if generated number is 16-digit

In [9]:
# inspect random sample of credit card numbers 
random.sample(credit_card_numbers, 5)

['4913642076996526',
 '4148299918528368',
 '4002894877268355',
 '4351659675456339',
 '4208821021263067']

# Generate credit card numbers

In [10]:
delta_time_object = datetime.datetime.strptime(START_DATE, DATE_FORMAT)
delta_time_object + datetime.timedelta(days=-728)

credit_cards = []
for cc_num in credit_card_numbers:
    credit_cards.append({'cc_num': cc_num, 'provider': 'visa', 'expires': faker.credit_card_expire(start=delta_time_object, end="+5y", date_format="%m/%y")})

In [11]:
credit_cards_pdf = pd.DataFrame.from_records(credit_cards)
credit_cards_pdf.head()

,cc_num,provider,expires
0,4796807885357879,visa,05/23
1,4529266636192966,visa,03/22
2,4922690008243953,visa,02/27
3,4897369589533543,visa,04/22
4,4848518335893425,visa,10/26


# Generate user identity 

In [12]:
profiles = []
for credit_card in credit_cards:
    address = faker.local_latlng(country_code = 'US')
    age = 0 
    profile = None
    while age < 18 or age > 100:
        profile = faker.profile(fields=['name', 'sex', 'mail', 'birthdate'])
        dday = profile['birthdate']
        delta = datetime.datetime.now() - datetime.datetime(dday.year, dday.month, dday.day)
        age = int(delta.days / 365)
    if age >= 18 or age <= 100:
        profile['City'] = address[2]
        profile['Country'] = address[3]
        profile['cc_num'] = credit_card['cc_num']
        profile['age'] = age
        credit_card['age'] = age
        profiles.append(profile)

In [13]:
profiles_pdf = pd.DataFrame.from_records(profiles)
profiles_pdf.drop('age', axis=1, inplace=True)
profiles_pdf.head()

,name,sex,mail,birthdate,City,Country,cc_num
0,Catherine Zimmerman,F,valenciajason@hotmail.com,1988-09-21,Huntley,US,4796807885357879
1,Michael Williams,M,brettkennedy@yahoo.com,1977-03-02,Gates-North Gates,US,4529266636192966
2,Jessica Krueger,F,marthacruz@hotmail.com,1947-09-11,Greenfield,US,4922690008243953
3,Ruth Harris,F,james11@yahoo.com,1983-12-28,New City,US,4897369589533543
4,Paul Ashley,M,matthew97@hotmail.com,1974-11-11,Peabody,US,4848518335893425


#### Generate Time Series


In [14]:
def generate_timestamps(n: int) -> list:
    start = datetime.datetime.strptime(START_DATE, DATE_FORMAT)
    end = datetime.datetime.strptime(END_DATE, DATE_FORMAT)
    timestamps = list()
    for _ in range(n):
        timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime(DATE_FORMAT)
        timestamps.append(timestamp)
    timestamps = sorted(timestamps)
    return timestamps

In [15]:
timestamps = generate_timestamps(TOTAL_UNIQUE_TRANSACTIONS)

In [16]:
assert len(timestamps) == TOTAL_UNIQUE_TRANSACTIONS

In [17]:
# inspect random sample of timestamps
random.sample(timestamps, 5)

['2022-02-27 13:51:28',
 '2022-01-22 12:22:21',
 '2022-01-27 11:25:06',
 '2022-01-14 22:11:46',
 '2022-01-20 11:40:26']

#### Generate Random Transaction Amounts 
<p>The transaction amounts are presumed to follow Pareto distribution, as it is logical for consumers to make many more smaller purchases than large ones. The break down of the distribution is shown in the table below.</p>


| Percentage        | Range (Amount in $)     |
| :-------------: | :----------: |
|  5\% | 0.01 to 1    |
| 7.5\%   | 1 to 10 |
| 52.5\%   | 10 to 100 |
| 25\%   | 100 to 1000 |
| 10\%   | 1000 to 10000 |

In [18]:
def get_random_transaction_amount(start: float, end: float) -> float:
    amt = round(np.random.uniform(start, end), 2)
    return amt

In [19]:
amounts = []
for percentage, span in AMOUNT_DISTRIBUTION_PERCENTAGES.items():
    n = int(TOTAL_UNIQUE_TRANSACTIONS * percentage)
    start, end = span
    for _ in range(n):
        amounts.append(get_random_transaction_amount(start, end+1))

categories = []        
for category, category_perc_price in CATEGORY_PERC_PRICE.items():
    percentage, min_price, max_price = category_perc_price
    n = int(TOTAL_UNIQUE_TRANSACTIONS * percentage)
    for _ in range(n):
        min_price_i = bisect.bisect_left(amounts, min_price)
        max_price_i = bisect.bisect_right(amounts, max_price, lo=min_price_i)
        categories.append({"category":category, "amount":random.choice(amounts[min_price_i:max_price_i])})
        
random.shuffle(categories)

In [20]:
len(categories)

54000

In [21]:
len(amounts)

54000

In [22]:
# inspect random sample of transaction amounts
random.sample(categories, 5)

[{'category': 'Grocery', 'amount': 16.4},
 {'category': 'Grocery', 'amount': 43.19},
 {'category': 'Grocery', 'amount': 37.13},
 {'category': 'Grocery', 'amount': 0.82},
 {'category': 'Grocery', 'amount': 80.37}]

#### Generate Credit Card Transactions
<br>
<div style="text-align: justify">
Using the random credit card numbers, timestamps and transaction amounts generated in the above steps, 
we can generate random credit card transactions by combining them. The transaction id for the transaction is the md5
hash of the above mentioned entities.
</div>

In [23]:
def generate_transaction_id(timestamp: str, credit_card_number: str, transaction_amount: float) -> str:
    hashable = f'{timestamp}{credit_card_number}{transaction_amount}'
    hexdigest = hashlib.md5(hashable.encode('utf-8')).hexdigest()
    return hexdigest

In [24]:
transactions = []
for timestamp, category in zip(timestamps, categories):
    credit_card_number = random.choice(credit_card_numbers)
    point_of_tr = faker.local_latlng(country_code = 'US')
    transaction_id = generate_transaction_id(timestamp, credit_card_number, category['amount'])
    transactions.append({
                         'tid': transaction_id, 
                         'datetime': timestamp, 
                         'cc_num': credit_card_number, 
                         'category': category['category'], 
                         'amount': category['amount'],
                         'latitude': point_of_tr[0], 
                         'longitude': point_of_tr[1],
                         'city': point_of_tr[2],
                         'country': point_of_tr[3],
                         'fraud_label': 0
                        }
                       )

In [25]:
# inspect random sample of credit card transactions
random.sample(transactions, 1)

[{'tid': '99883edc4d8aaa94b94b159865829379',
  'datetime': '2022-01-29 02:38:15',
  'cc_num': '4929411498746287',
  'category': 'Restaurant/Cafeteria',
  'amount': 53.95,
  'latitude': '44.99012',
  'longitude': '-123.02621',
  'city': 'Keizer',
  'country': 'US',
  'fraud_label': 0}]

#### Create Transaction Chains 

In [26]:
visited = set()
chains = defaultdict(list)

In [27]:
def size(chains: dict) -> int:
    counts = {key: len(values)+1 for (key, values) in chains.items()}
    return sum(counts.values())

In [28]:
def create_attack_chain(i: int):
    chain_length = random.choice(ATTACK_CHAIN_LENGTHS)
    for j in range(1, chain_length):
        if i+j not in visited:
            if size(chains) == NUMBER_OF_FRAUDULENT_TRANSACTIONS:
                break
            chains[i].append(i+j)
            visited.add(i+j)

In [29]:
while size(chains) < NUMBER_OF_FRAUDULENT_TRANSACTIONS:
    i = random.choice(range(TOTAL_UNIQUE_TRANSACTIONS))
    if i not in visited:
        create_attack_chain(i)
        visited.add(i)

## Create ATM cash withdrawal

In [30]:
cash_amounts = []
for percentage, span in AMOUNT_DISTRIBUTION_PERCENTAGES.items():
    n = int(TOTAL_UNIQUE_CASH_WITHDRAWALS * percentage)
    start, end = span
    for _ in range(n):
        cash_amounts.append(get_random_transaction_amount(start, end+1))

In [31]:
len(cash_amounts)

12000

In [32]:
def generate_atm_withdrawal(credit_card_number: str, cash_amounts: list, length: int, delta: int, radius: float = None, country_code = 'US') -> List[Dict]:
    atms = [] 
    start = datetime.datetime.strptime(START_DATE, DATE_FORMAT)
    end = datetime.datetime.strptime(END_DATE, DATE_FORMAT)
    timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None)
    point_of_tr = faker.local_latlng(country_code = country_code)
    latitude = point_of_tr[0] 
    longitude = point_of_tr[1]
    city = point_of_tr[2]
    for _ in range(length):
        current = timestamp + datetime.timedelta(hours=delta)
        if radius is not None:
            latitude = faker.coordinate(latitude, radius) 
            longitude = faker.coordinate(longitude, radius)  
        transaction_id = generate_transaction_id(timestamp, credit_card_number, category['amount'])
        atms.append({'tid': transaction_id, 
                     'datetime': current.strftime(DATE_FORMAT), 
                     'cc_num': credit_card_number, 
                     'category': 'Cash Withdrawal', 
                     'amount': random.sample(cash_amounts, 1)[0],
                     'latitude': latitude, 
                     'longitude': longitude,
                     'city': city,
                     'country': 'US',
                     'fraud_label': 0
                     })
        timestamp = current
    return atms

In [33]:
SUSCEPTIBLE_CARDS_DISTRIBUTION_BY_AGE = {
                                   0.055: (17, 24), 
                                   0.0015: (24, 34),
                                   0.0015: (34, 44),
                                   0.02: (44, 54),
                                   0.022: (54, 64),
                                   0.1: (64, 74),
                                   0.40: (74, 84),
                                   0.40: (84, 100),
                                  }
susseptible_cards = []
visited = []
for percentage, span in SUSCEPTIBLE_CARDS_DISTRIBUTION_BY_AGE.items():
    n = int(TOTAL_UNIQUE_CASH_WITHDRAWALS * percentage) ## TODO: here total expected fraud 
    start, end = span
    for _ in range(n):
        for card in credit_cards:
            if card['age'] > start and card['age'] < end:
                if card['cc_num'] not in visited:
                    current = card
                    visited.append(card['cc_num'])
                    break
                else:
                    current = None                    
        if current is not None:
            susseptible_cards.append(current)

In [34]:
normal_atm_withdrawals = []
atm_transactions = len(cash_amounts)
cash_withdrawal_cards = random.sample(susseptible_cards, CASH_WITHRAWAL_CARDS_TOTAL//(CASH_WITHRAWAL_CARDS_TOTAL//len(susseptible_cards)+1))
atm_count = 0
while atm_count < atm_transactions:
    for card in cash_withdrawal_cards:
        for ATM_WITHRAWAL_SEQ in ATM_WITHRAWAL_SEQ_LENGTH: 
            # interval in hours between normal cash withdrawals
            delta = random.randint(6, 168)
            atm_tr = generate_atm_withdrawal(credit_card_number = card['cc_num'], cash_amounts = cash_amounts, length=ATM_WITHRAWAL_SEQ, delta=delta, radius = NORMAL_ATM_RADIUS)         
            normal_atm_withdrawals.append(atm_tr)
            atm_count += ATM_WITHRAWAL_SEQ

In [35]:
len(normal_atm_withdrawals)

8000

#### Modify Transactions with Fraud Chain Attacks 

In [36]:
def generate_timestamps_for_fraud_attacks(timestamp: str, chain_length: int) -> list:
    timestamps = []
    timestamp = datetime.datetime.strptime(timestamp, DATE_FORMAT)
    for _ in range(chain_length):
        # interval in seconds between fraudulent attacks
        delta = random.randint(30, 120)
        current = timestamp + datetime.timedelta(seconds=delta)
        timestamps.append(current.strftime(DATE_FORMAT))
        timestamp = current
    return timestamps 

In [37]:
def generate_amounts_for_fraud_attacks(chain_length: int) -> list:
    amounts = []
    for percentage, span in AMOUNT_DISTRIBUTION_PERCENTAGES.items():
        n = math.ceil(chain_length * percentage)
        start, end = span
        for _ in range(n):
            amounts.append(get_random_transaction_amount(start, end+1))
    return amounts[:chain_length]

In [38]:
for key, chain in chains.items():
    transaction = transactions[key]
    timestamp = transaction['datetime']
    cc_num = transaction['cc_num']
    amount = transaction['amount']
    transaction['fraud_label'] = 1
    inject_timestamps = generate_timestamps_for_fraud_attacks(timestamp, len(chain))
    inject_amounts = generate_amounts_for_fraud_attacks(len(chain))
    random.shuffle(inject_amounts)
    for i, idx in enumerate(chain):
        original_transaction = transactions[idx]
        inject_timestamp = inject_timestamps[i]
        original_transaction['datetime'] = inject_timestamp
        original_transaction['fraud_label'] = 1
        original_transaction['cc_num'] = cc_num
        original_transaction['amount'] = inject_amounts[i]
        original_transaction['category'] = [category for category, category_perc_price in CATEGORY_PERC_PRICE.items() if int(inject_amounts[i]) in range(int(category_perc_price[1]), int(category_perc_price[2]))][0]
        original_transaction['tid'] = generate_transaction_id(inject_timestamp, cc_num, amount)
        transactions[idx] = original_transaction

#### Modify ATM normal cash withdrawals with fraudulent ones

In [39]:
fraudulent_atm_tr_indxs = random.sample([i for i in range(0, len(normal_atm_withdrawals))], int(FRAUD_RATIO * len(normal_atm_withdrawals)))

In [40]:
normal_atm_withdrawals[0]

[{'tid': 'ba3dc86c26230994301510dd4241e1bf',
  'datetime': '2022-02-26 05:14:58',
  'cc_num': '4978837587432787',
  'category': 'Cash Withdrawal',
  'amount': 66.79,
  'latitude': Decimal('27.107439'),
  'longitude': Decimal('-82.462260'),
  'city': 'Venice',
  'country': 'US',
  'fraud_label': 0},
 {'tid': '453c7ea434a3cbfeef5891a0eea429b8',
  'datetime': '2022-03-04 17:14:58',
  'cc_num': '4978837587432787',
  'category': 'Cash Withdrawal',
  'amount': 895.29,
  'latitude': Decimal('27.105581'),
  'longitude': Decimal('-82.454755'),
  'city': 'Venice',
  'country': 'US',
  'fraud_label': 0},
 {'tid': 'eaef30560442f424d97dee84a65f820a',
  'datetime': '2022-03-11 05:14:58',
  'cc_num': '4978837587432787',
  'category': 'Cash Withdrawal',
  'amount': 3779.88,
  'latitude': Decimal('27.102719'),
  'longitude': Decimal('-82.448723'),
  'city': 'Venice',
  'country': 'US',
  'fraud_label': 0}]

In [41]:
for fraudulent_atm_tr_indx in fraudulent_atm_tr_indxs:
    # interval in seconds between fraudulent attacks
    delta = random.randint(1, 5)
    atm_withdrawal = normal_atm_withdrawals[fraudulent_atm_tr_indx]
    pre_fraudulent_atm_tr = atm_withdrawal[0]
    fraudulent_atm_tr = generate_atm_withdrawal(credit_card_number = pre_fraudulent_atm_tr['cc_num'], cash_amounts = cash_amounts, length=1, delta=delta, radius = None)[0]
    fraudulent_atm_location = faker.location_on_land()
    while fraudulent_atm_location[3] == 'US':
        fraudulent_atm_location = faker.location_on_land()
    fraudulent_atm_tr['datetime'] = (datetime.datetime.strptime(pre_fraudulent_atm_tr['datetime'], DATE_FORMAT) + datetime.timedelta(hours=delta)).strftime(DATE_FORMAT)
    fraudulent_atm_tr['latitude'] = fraudulent_atm_location[0]
    fraudulent_atm_tr['longitude'] = fraudulent_atm_location[1]
    fraudulent_atm_tr['city'] = fraudulent_atm_location[2]
    fraudulent_atm_tr['country'] = fraudulent_atm_location[3]  
    fraudulent_atm_tr['fraud_label'] = 1 
    atm_withdrawal.append(fraudulent_atm_tr)
    normal_atm_withdrawals[fraudulent_atm_tr_indx] = atm_withdrawal

In [42]:
for atm_withdrawal in normal_atm_withdrawals:
    for withdrawal in atm_withdrawal:
        transactions.append(withdrawal)  

### Produce transactions 

In [43]:
from hops import kafka
from hops import tls
from hops import hdfs
import json
from confluent_kafka import Producer

## TODO (davit): gif how to create kafka topic from the UI

In [44]:
# change this according to your settings
KAFKA_BROKER_ADDRES = "broker.kafka.service.consul:9091"
KAFKA_TOPIC_NAME = "credit_card_transactions"

In [45]:
config = {
    "bootstrap.servers": KAFKA_BROKER_ADDRES,
    "security.protocol": kafka.get_security_protocol(),
    "ssl.ca.location": tls.get_ca_chain_location(),
    "ssl.certificate.location": tls.get_client_certificate_location(),
    "ssl.key.location": tls.get_client_key_location(),
    "group.id": "1"
}

producer = Producer(config)

In [ ]:
i = 0
for transaction in transactions:
    transaction_label = {}
    _ = transaction.pop("fraud_label")
    if i % 1000 == 0:
        print(json.dumps(transaction))
    producer.produce(KAFKA_TOPIC_NAME, json.dumps(transaction))
    producer.flush()    
    i += 1

### Next Steps

In the next notebook, we'll look at how to do streaming aggregations using Spark structured streaming.